In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/home/beans/bespoke')

from constants import *
from imports import *

In [5]:
for i in range(100):
    paths = glob.glob(f"{BLENDER_MEMBANK_ROOT}/**/obs_per_sec.npy", recursive=True)
    obs_per_sec = np.array([np.load(p)[0] for p in paths])
    print(obs_per_sec)
    time.sleep(60*1)

[]
[6.276742  6.9023676 6.274891  6.560123 ]
[5.8222213 5.608184  5.6848035 5.2308936 6.276742  5.3758264 5.275483
 5.8369913 6.9023676 6.274891  5.456125  6.560123 ]
[5.8222213 5.608184  5.6848035 5.2308936 6.276742  5.3758264 5.275483
 5.8369913 6.9023676 6.274891  5.456125  6.560123 ]
[8.52803   5.608184  8.86775   5.2308936 8.690125  8.373009  8.943278
 8.105022  7.8128724 6.5507092 5.456125  6.916938 ]
[8.52803   6.223542  8.86775   6.0000253 8.690125  8.373009  8.943278
 8.105022  7.8128724 6.5507092 5.311092  6.916938 ]
[8.52803   6.223542  9.16094   6.0000253 8.540795  8.373009  8.943278
 9.070953  7.8128724 9.446785  5.311092  6.916938 ]
[6.6949844 6.223542  9.16094   8.691761  8.540795  6.240204  5.975263
 9.070953  5.978112  9.446785  5.311092  7.6038175]
[6.6949844 6.3109093 8.27924   8.691761  8.540795  6.240204  5.975263
 9.070953  5.978112  9.458635  6.5549726 7.6038175]
[8.172321  6.3109093 8.27924   8.691761  6.5685177 6.240204  5.975263
 7.4976015 8.872364  9.458635  

[7.6969094 7.6078715 6.820962  6.810355  5.999622  6.7134266 6.601609
 7.562371  5.891546  9.024759  7.593692  7.4260783]
[7.6969094 7.6078715 5.878294  6.810355  5.999622  6.7134266 6.601609
 7.562371  5.891546  9.024759  7.593692  7.4260783]
[6.2984343 5.712943  5.878294  6.810355  5.1811185 5.4068217 6.601609
 5.4766154 5.891546  9.139635  5.5640574 7.4260783]
[6.2984343 5.712943  5.878294  5.8048587 5.1811185 5.4068217 5.2164116
 5.4766154 6.425748  9.139635  5.5640574 6.850224 ]
[6.2984343 5.712943  6.489399  5.8048587 5.1811185 5.4068217 5.2164116
 6.740951  6.425748  9.139635  8.852432  6.850224 ]
[6.2984343 5.712943  6.489399  5.8048587 6.3714495 6.55145   5.2164116
 6.740951  6.425748  8.377228  8.852432  6.850224 ]


KeyboardInterrupt: 

In [7]:
(420/480)*1080

945.0

In [8]:
a = np.random.randn(32, 9, 120, 640, 3)
a.shape

(32, 9, 120, 640, 3)

In [9]:
np.mean(a, -1).shape

(32, 9, 120, 640)

In [17]:
SEQ_LEN

580

In [10]:
set_should_stop(True)

In [11]:
set_lr(1e-4)

In [8]:
import numpy as np

In [9]:
a = np.array([1,2,3,4,1,2,3])

In [11]:
np.where(a==1)[0]

array([0, 4])

In [12]:
max(np.nan, 0)

nan

In [13]:
np.clip(12, 10,11)

11